In [ ]:
import psycopg2
import pandas as pd
import configparser as configparser
import os

#%matplotlib inline###########\
import pandas as pd
import os

import numpy as np
import seaborn as sns
import subprocess
import time

from scipy.stats import ttest_ind, mannwhitneyu

#save plot without displaying
#https://stackoverflow.com/questions/3285193/how-to-switch-backends-in-matplotlib-python
import matplotlib
matplotlib.use("Agg", warn=False, force=True)
import matplotlib.pyplot as plt


import glob
import sys
import collections
import fcsparser

import dask
from distributed import Client
from pathlib import Path

In [37]:
class PostgresConnection:
    """Establishes a connection to a various DB using credentials in config.txt,
     to push and pull data to data stores
    """
    def __init__(self, inputPath):
        config = configparser.ConfigParser()
        config.read(os.path.join(inputPath,  "config.ini"))
        self.host = config.get('psql database', 'host')
        self.db_name = config.get('psql database', 'db_name')
        self.user = config.get('psql database', 'user')
        self.password = config.get('psql database', 'password')
        self.port = config.get('psql database', 'port')

    def get_db_data(self, sql_string, data_frame_it=False):
        """Queries the DB specified when instantiating the PostgresConnection class, and converts to a pandas dataframe.
        Args:
            sql_string (str): the SQL query to execute
            data_frame_it (bool): default false, returns dataframe if true
        Returns:
            raw result:
            pandas dataframe: the results of the query, using the column headers provided in the DB
        """
        connection_string = f"""
        host='{self.host}'
        dbname='{self.db_name}'
        user='{self.user}'
        password='{self.password}'
        port='{self.port}'
        """

        with psycopg2.connect(connection_string) as connection:
            cursor = connection.cursor()
            cursor.execute(sql_string)
            
            # two modes raw and data framed
            if data_frame_it == True:
                result = pd.DataFrame(cursor.fetchall())
                result.columns = [desc[0] for desc in cursor.description]
            else:
                result = cursor.fetchall()
            
        return result

    def set_db_data(self, sql_strings):
        """Writes to the DB specified when instantiating the PostgresConnection class using the supplied SQL string.
        Args:
            sql_strings (:obj:`list` of :obj:`str`): list of SQL strings to execute
        """
        connection_string = f"""
        host='{self.host}'
        dbname='{self.db_name}'
        user='{self.user}'
        password='{self.password}'
        port='{self.port}'
        """

        with psycopg2.connect(connection_string) as connection:
            cursor = connection.cursor()
            for query in sql_strings:
                cursor.execute(query)

            connection.commit()
    
    def find_open_analysis(self, sql_strings):
        
        reply=self.get_db_data(sql_strings)
        
        print (reply)
        
    def update_run_data(self, sql_strings):
        pass
    def start_analysis(self, runparams):
        pass  
    def push_cytobank_meta(self, sql_strings):
        pass
    def push_cytobank_fcs(self, sql_strings):    
        pass

In [38]:
# mount the shared drive smb://45.55.42.213/data
#import os
#directory = "/Users/user.name/foldername"
#if not os.path.exists(directory): os.makedirs(directory)
#os.system("mount_smbfs //user.name:shareuser@server/servershare ~/foldername")

#os.system("mount_smbfs //user.name:shareuser@45.55.42.213/data")

In [39]:
#!/usr/bin/python
from configparser import ConfigParser
 
 
def config(filename='../input/config.ini', section='psql database'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

conn_string=config()
conn = psycopg2.connect(**conn_string) # pass the connection string in from the file
#cursor = conn.cursor()



OperationalError: could not connect to server: Operation timed out
	Is the server running on host "reddaanalyticspg.cngkcakawy89.us-west-1.rds.amazonaws.com" (10.113.29.48) and accepting
	TCP/IP connections on port 5432?


# A place to see the run parameters

7="../input/config.ini" 11L, 193C▽  [>c]11;?[psql database]
host=reddaanalyticspg.cngkcakawy89.us-west-1.rds.amazonaws.com
database=postgress
db_name=postgress
user=drozelle
password=fgh456
port=5432

[PipelinePaths]

[Archival Storage]
~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                     

In [15]:
#Conn.__dict__
Conn=PostgresConnection('../input')
Conn.get_db_data("SELECT *")

#conn = psycopg2.connect(conn_string)
#cursor = Conn.cursor()
#cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
#print cursor.fetchall()

OperationalError: could not connect to server: Operation timed out
	Is the server running on host "reddaanalyticspg.cngkcakawy89.us-west-1.rds.amazonaws.com" (10.113.29.48) and accepting
	TCP/IP connections on port 5432?


In [19]:
#a dictionary storing the data tables
Conn=PostgresConnection('../input')
tb_Ls = ['analysis_run','analysis_run_parameters',\
'fcsfiles','sessions', 'treatments', 'rundata']

tb_dict={}
# make a dictionary with all the tables.
tb_dict={x:Conn.get_db_data("SELECT * from postgress.{};".format(x)) for x in tb_Ls}
tb_dict.keys()




OperationalError: could not connect to server: Operation timed out
	Is the server running on host "reddaanalyticspg.cngkcakawy89.us-west-1.rds.amazonaws.com" (10.113.29.48) and accepting
	TCP/IP connections on port 5432?


In [6]:
Conn.get_db_data("SELECT * from cytofui.analysis_run;", True)

,analysis_run_id,session_id,analysis_run_name,analysis_run_parameter,selected_fcs_files,run_state,description,analysis_run_timestamp
0,1,1,1_201802221411,None,[],0,1_201802221411,2018-04-09 13:11:57.333997
1,54,35,35_201803270601,None,[],3,35_201803270601,2018-04-09 13:11:57.333997
2,98,48,48_98,"{""Data_Transformation"":[""_"",""cytofAsinh"",""_"",""...","[""20170724_Spleen_001_Tx7_1_hu_4_smithj_1.fcs""...",3,Test3,2018-04-13 03:14:43.540486
3,94,35,35_94,"{""Data_Transformation"":[""_"",""cytofAsinh"",""_"",""...","[""c02_Ms SP Tx1_001_1.fcs"",""c02_Ms SP Tx1_002_...",3,This is a test run,2018-04-12 14:35:27.936589
4,46,40,40_201803231353,"{""Data_Transformation"":[""_"",""cytofAsinh"",""_"",""...","[""on"",""txt_parent_001_1.fcs"",""txt_parent_002_0...",99992,40_201803231353,2018-04-09 13:11:57.333997
5,14,19,19_201803022024,None,[],0,19_201803022024,2018-04-09 13:11:57.333997
6,92,40,40_92,"{""Data_Transformation"":[""_"",""cytofAsinh"",""_"",""...","[""txt_parent_001_1.fcs"",""txt_parent_002_0_1.fcs""]",9999,New Run,2018-04-09 13:14:16.661040
7,16,17,17_201803060556,"{""Data_Transformation"":[""_"",""cytofAsinh"",""_"",""...","[""fcs001.fcs"",""fcs002.fcs"",""fcs003.fcs""]",99990,17_201803060556,2018-04-09 13:11:57.333997
8,25,25,25_201803132021,None,[],0,25_201803132021,2018-04-09 13:11:57.333997
9,24,25,25_201803132020,None,[],0,25_201803132020,2018-04-09 13:11:57.333997


In [7]:
# Reset the run state so that it is no longer listed as finshed
runlist=['15_201803010624','14_201803010351', '14_201802282259', '20_201803022045', '16_201803011530']
for RunID in runlist:
    Conn.set_db_data(["UPDATE cytofui.analysis_run SET run_state=1 where analysis_run_name ='{}';".format(RunID)])
            

# This is the run parameter object

In [26]:
Conn.get_db_data("SELECT analysis_run_parameter from cytofui.analysis_run where analysis_run_name ='16_201803011530';")

[('{"Data_Transformation":["_","autoLgcl","_","logicile","_","_"],"comparisonParameter":{"groupings":"Tx7 vs. Tx8","dimension_variable":"treatment_type_control"},"filterMarkersRemoved":["_",["Pd102Di<102Pd>","Pd105Di<105Pd>","Pd108Di<108Pd>","Xe124Di<124Xe>","Cs133Di<133Cs>","La138Di<138La>","Ce140Di<140Ce_Bead>","Nd142Di<142Nd_CD19>","Nd144Di<144Nd_KLRG1>","Nd146Di<146Nd_FOXP3>","Nd148Di<148Nd_CD16>","Nd150Di<150Nd_CCR7>","Sm152Di<152Sm_CD62L>","Sm154Di<154Sm_CD3>","Gd156Di<156Gd_CCR2>","Tb159Di<159Tb_PDL1>","Dy161Di<161Dy_41BB>","Dy164Di<164Dy_CD15>","Er166Di<166Er_CD141>","Er168Di<168Er_CD8>","Er170Di<170Er_GITR>","Yb172Di<172Yb_CD103>","Yb174Di<174Yb_HLADR>","Yb176Di<176Yb_CD127>"],"_",["Pd102Di<102Pd>","Pd105Di<105Pd>","Pd108Di<108Pd>","Xe124Di<124Xe>","Cs133Di<133Cs>","La138Di<138La>","Ce140Di<140Ce_Bead>","Nd142Di<142Nd_CD19>","Nd144Di<144Nd_KLRG1>","Nd146Di<146Nd_FOXP3>","Nd148Di<148Nd_CD16>","Nd150Di<150Nd_CCR7>","Sm152Di<152Sm_CD62L>","Sm154Di<154Sm_CD3>","Gd156Di<156Gd_CCR2>

In [27]:
Conn.get_db_data("SELECT selected_fcs_files from cytofui.analysis_run WHERE analysis_run_name ='1_201802240057';")

[('["fcs001.fcs","fcs002.fcs","fcs003.fcs","fcs004.fcs","fcs005.fcs","fcs006.fcs"]',)]

In [11]:
Conn.get_db_data("SELECT * from cytofui.analysis_run;", True)

,analysis_run_id,session_id,analysis_run_name,analysis_run_parameter,selected_fcs_files,run_state
0,1,1,1_201802221411,None,[],0
1,2,1,1_201802221519,None,[],0
2,3,1,1_201802221651,None,[],0
3,4,1,1_201802221823,None,[],0
4,9,11,11_201802282226,None,[],0
5,6,4,4_201802280347,None,[],0
6,7,4,4_201802280347,None,[],0
7,8,4,4_201802280347,None,[],0
8,5,1,1_201802240057,"{""Data_Transformation"":[""_"",""cytoAsinh"",""_"",""c...","[""fcs001.fcs"",""fcs002.fcs"",""fcs003.fcs"",""fcs00...",0
9,10,14,14_201802282259,"{""Data_Transformation"":[""_"",""cytofAsinh"",""_"",""...","[""fcs001.fcs"",""fcs002.fcs"",""fcs003.fcs""]",3


In [24]:
Conn.get_db_data("SELECT * from cytofui.sessions;", True)

,session_id,session_name,state,experiment_properties_set,output_ready
0,1,Test Session,0,0,0
1,2,Test Session 2,0,0,0
2,3,Test session 3,0,0,0
3,4,session 25,0,0,0
4,5,Session 7,0,0,0
5,6,Session 20180228,0,0,0
6,7,10,0,0,0
7,8,Session10,0,0,0
8,9,Session 201802280458,0,0,0
9,10,ATSESSION01,0,0,0


In [20]:
Conn=PostgresConnection('../input')
Conn.get_db_data("SELECT * from cytofui.treatments;", True)

,treatment_id,treatment_type,molecule_name,molecule,number_of_days,dosage,dosage_unit,session_id,global_record
0,16,Tx2,Butadopanolamine,pfz1,15,0.5,mg/kg,1,False
1,18,Tx4,Butadopanolamine,pfz1,15,0.5,mg/kg,1,False
2,19,Tx5,Butadopanolamine,pfz1,,0.5,mg/kg,1,False
3,20,Tx6,Butadopanolamine,pfz1,,0.5,mg/kg,1,False
4,21,Tx7,Butadopanolamine,pfz1,,0.5,mg/kg,1,False
5,22,Tx8,Butadopanolamine,pfz1,,0.5,mg/kg,1,False
6,23,Tx10,Butadopanolamine,pfz1,,0.5,mg/kg,1,False
7,4,Tx1,Butadopanolamine,pfz1,15,0.5,mg/kg,1,False
8,17,Tx3,Butadopanolamine,pfz1,15,0.5,g/kg,1,False
9,9,,Butadopanolamine,pfz1,,0.5,mg/kg,1,True


In [21]:
# should global vlaules be global default since it is often overwritten?

#FcsTable=Conn.get_db_data("SELECT * from cytofui.fcsfiles;", True)
Conn.get_db_data("SELECT * from cytofui.fcsfiles;", True)

,fcs_id,session_id,fcs_filename,sample_id,patient_id,plate_row,plate_column,plate,human_disease,animal_model_genotype,...,replicate_no,plate_shape,global_record,nsample_id,file_exists,sample_barcoding,sample_ic_strain,sample_storage,straining_format,batch_number
0,14,1,fcs014.fcs,14,014,,,,Cancer,B6,...,,,False,14,False,None,None,None,None,None
1,15,1,fcs015.fcs,15,015,,,,Cancer,B6,...,,,False,15,False,None,None,None,None,None
2,20,3,fcs001.fcs,1,001,,,,Cancer,B6,...,,,False,1,False,None,None,None,None,None
3,21,3,fcs002.fcs,2,002,,,,Cancer,B6,...,,,False,2,False,None,None,None,None,None
4,22,3,fcs003.fcs,3,003,,,,Cancer,B6,...,,,False,3,False,None,None,None,None,None
5,1,1,fcs001.fcs,1,001,G,Y,,Cancer,B6,...,,,False,1,False,Pre-Ab Pd-Barcoding,Nuclear,MeOH (-20*C),96-deep well,
6,0,1,,Global Values,,,,,Cancer,B6,...,,,True,0,False,Pre-Ab Pd-Barcoding,Nuclear,MeOH (-20*C),96-deep well,
7,7,1,fcs007.fcs,7,007,,,,Cancer,B6,...,,,False,7,False,None,None,None,None,None
8,25,6,global metadata,None,None,None,,None,None,None,...,None,None,True,0,False,None,None,None,None,None
9,23,4,global metadata,None,None,None,,None,None,None,...,None,None,True,0,False,None,None,None,None,None


In [17]:
# the table is not totally filled, 
# what is run parameter ID 
# why are there doubles in analysis run ID/ session ID
# there should be a process that foces the completion 
#of all fileds if methos is completed

sql_string="SELECT * from cytofui.analysis_run_parameters;"
Conn.get_db_data(sql_string, True)


,analysis_run_parameter_id,analysis_run_id,session_id,method,fcs_concatination,transformation,smallest_file_size,fixed_number,k_parameter,parallel,setseed,dimension_reductions,cores,filter_markers,filter_markers_removed
0,2,1,1,Rphenograph,,,,,,,,,None,None,[]
1,4,2,1,ClusterX,min,,100,,,,,,None,None,[]
2,5,3,1,ClusterX,min,,,,,,,,None,None,[]
3,7,3,1,Rphenograph,ceil,,,,,,,,None,None,[]
4,1,1,1,ClusterX,min,,100,100,,,,,,[],[]
5,8,3,1,Rphenograph,all,,,,,,,,,[],[]
6,6,4,1,ClusterX,all,,,,,,,,,"[""Y89Di<89Y_CD45>"",""Pd102Di<102Pd>"",""Pd104Di<1...",[]
7,12,4,1,Rphenograph,all,,100,,,,,,,,[]
8,28,5,1,FlowSOM,all,cytoAsinh,100,100,100,Yes,1234,pca,,"[""Y89Di<89Y_CD45>"",""Pd102Di<102Pd>"",""Pd104Di<1...","[""Pd102Di<102Pd>"",""Pd105Di<105Pd>"",""Pd108Di<10..."
9,29,10,14,Rphenograph,all,cytoAsinh,100,100,15,,1234,tsne,,"[""Y89Di<89Y_CD45>"",""Pd102Di<102Pd>"",""Pd104Di<1...","[""Pd102Di<102Pd>"",""Pd105Di<105Pd>"",""Pd108Di<10..."


In [21]:
os.getcwd()

'/Users/GunnarK/Desktop/gunner/CYTOF/gui_build/Analysis_folder/scripts'